<a href="https://colab.research.google.com/github/Ngocc508/Tri-Tue-Nhan-Tao-/blob/main/THTTNT_Buoi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Nguyễn Thị Ngọc-2001230571
## MINIMAX

**1. Định nghĩa**

Minimax là một thuật toán đệ quy được sử dụng trong lý thuyết trò chơi và trí tuệ nhân tạo cho các trò chơi có tổng bằng không (zero-sum games) với hai người chơi.

*   MAX (Máy tính - Người chơi tối đa hóa): Cố gắng chọn nước đi để đạt điểm số cao nhất (thắng).

*  MIN (Đối thủ - Người chơi tối thiểu hóa): Cố gắng chọn nước đi để điểm số của MAX là thấp nhất (làm MAX thua hoặc hòa).

**2. Cơ chế hoạt động**

Thuật toán xây dựng một "Cây trò chơi" (Game Tree) từ trạng thái hiện tại. Nó đi sâu xuống các trạng thái tương lai (giả lập các nước đi của cả 2 bên) cho đến khi đạt đến Trạng thái kết thúc (thắng/thua/hòa) hoặc đạt độ sâu giới hạn. Sau đó, nó "truy ngược" (backtrack) giá trị từ dưới lên trên:

1.  Tại lượt của MAX, giá trị của nút cha là giá trị lớn nhất của các nút con.
2.   Tại lượt của MIN, giá trị của nút cha là giá trị nhỏ nhất của các nút con.

**3. Cách chạy thuật toán (Ví dụ minh họa)**

Giả sử chúng ta đang ở một trạng thái bàn cờ, và thuật toán cần quyết định nước đi cho X (MAX).

*- Bước 1 (Tạo cây):* Từ trạng thái hiện tại, tạo ra tất cả các trạng thái con (các ô trống có thể đánh). Tiếp tục làm như vậy cho lượt của O (MIN), rồi lại đến X (MAX)... cho đến khi trò chơi kết thúc (tìm thấy người thắng hoặc hòa).

*- Bước 2 (Đánh giá nút lá - Terminal State):* Gán điểm cho các trạng thái kết thúc:
*   Nếu X thắng: +10 điểm.
*   Nếu O thắng: -10 điểm.
*   Nếu Hòa: 0 điểm.
*   (Lưu ý: Nếu dùng hàm Heuristic $E(n) = X(n) - O(n)$ như đề bài, ta sẽ tính toán giá trị này tại các nút lá hoặc nút giới hạn độ sâu thay vì chỉ +10/-10).

*- Bước 3 (Truy ngược - Backpropagation):*
Từ các nút lá, đưa giá trị lên tầng trên.
Nếu tầng trên là lượt của MIN (O): O sẽ chọn nước đi có điểm thấp nhất (ví dụ giữa 10 và -10, O sẽ chọn đường dẫn đến -10).

Nếu tầng trên là lượt của MAX (X): X sẽ chọn nước đi có điểm cao nhất.

*- Bước 4 (Ra quyết định):* Tại nút gốc (trạng thái hiện tại), MAX sẽ chọn nhánh dẫn đến giá trị lợi ích cao nhất đã được truyền lên.

**3. Triển khai Code (Python)**

Trong ngữ cảnh trò chơi Cờ Caro tổng quát (bao gồm Tic-Tac-Toe và Gomoku), Minimax là giải thuật tìm kiếm nước đi tối ưu trong không gian trạng thái.
- Đối với bàn cờ nhỏ ($3 \times 3$): Thuật toán có khả năng duyệt toàn bộ cây trò
chơi (full state-space search) đến tận trạng thái kết thúc để đưa ra quyết định chính xác tuyệt đối.
- Đối với bàn cờ lớn ($N \times N$): Do sự bùng nổ tổ hợp (combinatorial explosion), thuật toán buộc phải giới hạn độ sâu tìm kiếm (depth-limited search). Khi đó, Minimax sử dụng hàm lượng giá Heuristic để ước lượng tiềm năng của các trạng thái trung gian thay vì đi đến trạng thái kết thúc thực sự.

In [ ]:
import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

*Các hàm xử lý trạng thái*

---
Nhóm hàm này chịu trách nhiệm khởi tạo bàn cờ, xác định lượt đi, tìm các nước đi hợp lệ và cập nhật bàn cờ sau khi đi.

In [ ]:
def initial_state(n):
    """
    Returns starting state of the board.
    """
    return [[EMPTY for _ in range(n)] for _ in range(n)]

def player(board):
    """
    Returns player who has the next turn on a board.
    """
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1
    if count % 2 != 0:
        return ai
    return user

def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

*Các hàm kiểm tra kết quả và luật chơi*

---

Nhóm hàm này kiểm tra xem ai thắng (ngang, dọc, chéo), game đã kết thúc chưa (terminal) và chấm điểm bàn cờ (utility).

In [ ]:
def get_horizontal_winner(board):
    # check horizontally
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[i][0]
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val
    return winner_val

def get_vertical_winner(board):
    # check vertically
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[0][i]
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val
    return winner_val

def get_diagonal_winner(board):
    # check diagonally
    winner_val = None
    board_len = len(board)

    # Check diagonal 1
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
    if winner_val:
        return winner_val

    # Check diagonal 2
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None
    return winner_val

def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
    return winner_val

def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) != None:
        return True
    for i in board:
        for j in i:
            if j == EMPTY:
                return False
    return True

def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

*Thuật toán Minimax*

---
Sử dụng đệ quy (maxValue và minValue) để tìm nước đi tối ưu nhất.

In [ ]:
def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    current_player = player(board)

    if current_player == X:
        min_val = -math.inf
        move = None
        for action in actions(board):
            check = minValue(result(board, action))
            if check > min_val:
                min_val = check
                move = action
    else:
        max_val = math.inf
        move = None
        for action in actions(board):
            check = maxValue(result(board, action))
            if check < max_val:
                max_val = check
                move = action
    return move

*Chương trình chính*

In [ ]:
if __name__ == "__main__":
    # Khởi tạo bàn cờ
    n = int(input("Enter the size of the board: "))
    board = initial_state(n)
    ai_turn = False

    print("Choose a player (X or O):")
    user = input()

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            winner_is = winner(board)
            if winner_is is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {winner_is} wins.")
            break

        else:
            if user != playr and not game_over:
                # Lượt của AI
                if ai_turn:
                    print("AI is thinking...")
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    print(numpy.array(board))

            elif user == playr and not game_over:
                ai_turn = True
                print(numpy.array(board))
                print("Enter the position to move (row,col)")
                try:
                    i = int(input(f"Row (0-{n-1}): "))
                    j = int(input(f"Col (0-{n-1}): "))
                    if board[i][j] == EMPTY:
                        board = result(board, (i, j))
                        print(numpy.array(board))
                    else:
                        print("Spot already taken!")
                        ai_turn = False # Cho nhập lại
                except (ValueError, IndexError):
                    print("Invalid input! Please enter numbers 0-2.")
                    ai_turn = False2

### ALPHA-BETA


**1. Tổng quan**

Cắt tỉa Alpha-Beta là một kỹ thuật tối ưu hóa cho thuật toán Minimax. Thay vì duyệt toàn bộ cây trò chơi (vốn tốn rất nhiều tài nguyên và thời gian), thuật toán sẽ "cắt bỏ" (pruning) các nhánh không cần thiết – tức là những nhánh mà tại đó, máy tính đã biết chắc chắn rằng kết quả sẽ tồi tệ hơn hoặc không tốt bằng các phương án đã tìm thấy trước đó

**2. Các tham số chính**

Thuật toán duy trì hai giá trị trong suốt quá trình tìm kiếm trên cây:


*  $\alpha$ (Alpha) - Giá trị của MAX:
    - Đại diện cho giá trị tốt nhất (cao nhất) mà người chơi MAX đã tìm thấy cho đến thời điểm hiện tại dọc theo đường đi đến nút gốc.
    - Giá trị khởi tạo: $-\infty$.
    - Chỉ cập nhật tại các nút MAX.
    - Giá trị $\alpha$ không bao giờ giảm.
*  $\beta$ (Beta) - Giá trị của MIN:
    - Đại diện cho giá trị tốt nhất (thấp nhất) mà người chơi MIN đã tìm thấy cho đến thời điểm hiện tại.
    - Giá trị khởi tạo: $+\infty$.
    - Chỉ cập nhật tại các nút MIN.
    - Giá trị $\beta$ không bao giờ tăng.

**3. Nguyên lý hoạt động**
Quy tắc vàng của thuật toán là: Cắt tỉa khi $\alpha \ge \beta$.

**Giải thích logic:**
* Nếu bạn đang ở một nút con và tìm thấy một nước đi tồi tệ đến mức đối thủ (MIN) chắc chắn sẽ ép bạn vào đó (giá trị $\beta$ của MIN nhỏ hơn hoặc bằng giá trị $\alpha$ mà bạn (MAX) đã có ở nhánh khác), thì bạn không cần xét thêm bất kỳ con nào khác của nhánh hiện tại nữa.
* Người chơi MAX sẽ không bao giờ chọn đi vào nhánh này vì đã có một lựa chọn tốt hơn ($\alpha$) ở nhánh trước đó.


**4. Ví dụ minh họa - Trò chơi cờ caro (TicTacToe)**

Trong ngữ cảnh ứng dụng cho Tic-Tac-Toe (và các game đối kháng nói chung), kỹ thuật này đảm bảo kết quả tìm được là tối ưu giống hệt Minimax nhưng với hiệu suất cao hơn.
- **Chiến lược Thắng nhanh - Thua chậm**: Mã nguồn cài đặt sử dụng độ sâu (depth) làm trọng số trong hàm lượng giá. Thắng ở độ sâu thấp (+10 - depth) được ưu tiên hơn thắng ở độ sâu cao; ngược lại, thua ở độ sâu cao (-10 + depth) được chấp nhận hơn thua ngay lập tức.
- **Hiệu quả tìm kiếm**: Với Tic-Tac-Toe, Alpha-Beta có thể cắt giảm đáng kể số lượng node phải duyệt, đặc biệt ở các nước đi đầu tiên khi cây trò chơi còn rất rộng.


*Khởi tạo và Các hàm tiện ích*

In [ ]:
import math
import copy

Hàm đệ quy thực hiện việc duyệt cây, tính điểm và cắt tỉa nhánh thừa.

In [ ]:
def _alpha_beta_recursive(board, depth, alpha, beta, is_max_player):
     if terminal(board):
        score = utility(board)
        if score == 1:
            return score + (10 - depth)
        elif score == -1:
            return score - (10 - depth)
        else:
            return score

    if is_max_player:
        max_eval = -math.inf
        for action in actions(board):
            new_board = result(board, action)
                eval = _alpha_beta_recursive(new_board, depth + 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
            for action in actions(board):
            new_board = result(board, action)

            eval = _alpha_beta_recursive(new_board, depth + 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

Hàm quyết định

In [ ]:
def FindBestMove(board, ai_player_char):
    """
    Tìm nước đi tối ưu nhất cho người chơi AI bằng Alpha-Beta Pruning.
    Sử dụng board 2D và các hàm game logic từ phần Minimax.
    """
    best_move = None
    current_ai_player = ai_player_char

    if current_ai_player == X:
        best_val = -math.inf
        for action in actions(board):
            new_board = result(board, action)
            move_val = _alpha_beta_recursive(new_board, 0, -math.inf, math.inf, False)
            if move_val > best_val:
                best_val = move_val
                best_move = action
    else:
        best_val = math.inf

        for action in actions(board):
            new_board = result(board, action)
            move_val = _alpha_beta_recursive(new_board, 0, -math.inf, math.inf, True)
            if move_val < best_val:
                best_val = move_val
                best_move = action

    return best_move

Chương trình chính

In [ ]:
import numpy as np
if __name__ == "__main__":
    n = int(input("Enter the size of the board (e.g., 3): "))

    board = initial_state(n)

    print("Choose a player (X or O):")
    while True:
        user = input().strip().upper()
        if user in ["X", "O"]:
            break
        print("Invalid choice. Please enter X or O.")

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    print(f"\nGame Start! User: {user} vs AI: {ai}")
    print(np.array(board))

    while True:
        game_over = terminal(board)
        current_player = player(board)

        if game_over:
            print("\n-------------------")
            print(np.array(board))
            winner_is = winner(board)
            if winner_is is None:
                print(">>> Game Over: Tie.")
            else:
                print(f">>> Game Over: {winner_is} wins.")
            break

        if current_player != user:
            print(f"\n[{ai}] AI is thinking...")


            move = FindBestMove(board, ai)

            if move is None:
                print("AI cannot find a move (Error).")
                break

            board = result(board, move)
            print(f"AI chose: {move}")
            print(np.array(board))

        else:
            print(f"\n[{user}] Your turn.")
            while True:
                try:
                    i = int(input(f"Row (0-{n-1}): "))
                    j = int(input(f"Col (0-{n-1}): "))

                    if 0 <= i < n and 0 <= j < n:
                        if board[i][j] == EMPTY:
                            board = result(board, (i, j))
                            print(np.array(board))
                            break
                        else:
                            print("Spot already taken! Try again.")
                    else:
                        print(f"Coordinates must be between 0 and {n-1}.")

                except ValueError:
                    print("Invalid input! Please enter integers.")